# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import TruncatedSVD 
from lightgbm import LGBMClassifier

import pickle
import bz2

from sqlalchemy import create_engine
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM disaster_response', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df['message']
Y = df.drop(['id','message','original','genre'],axis=1)

### 2. Write a tokenization function to process your text data

In [6]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Han\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Han\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Han\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def tokenize(text):
    # normalize text and strip punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    token_list = word_tokenize(text) 
    token_list = [w for w in token_list if w not in stopwords.words("english")]
    
    # lemmatize and return token list
    token_list = [WordNetLemmatizer().lemmatize(w) for w in token_list]
    return token_list

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
from lightgbm import LGBMClassifier

def pipeline_lgbClassifier():
    pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('Multi_classifier',MultiOutputClassifier(LGBMClassifier(device='gpu',
                                                                                 gpu_platform_id = 0, gpu_device_id = 0)))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [13]:
pipeline = pipeline_lgbClassifier()
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                gpu_device_id=0,
                                                                gpu_platform_id=0,
                                                                importance_type='split',
                                                                learning_rate=0.1,

In [14]:
import pickle

pickle_out = open('../models/pipeline_previous_model.pickle','wb')
pickle.dump(pipeline, pickle_out)
pickle_out.close()

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
y_pred = pipeline.predict(X_test)        

In [16]:
target_name_list = df.columns[4:].tolist()

In [17]:
for i,col in enumerate(target_name_list):
    print(col)
    print(classification_report(y_test[col].values, y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.69      0.47      0.56      2118
           1       0.85      0.93      0.89      6534

    accuracy                           0.82      8652
   macro avg       0.77      0.70      0.73      8652
weighted avg       0.81      0.82      0.81      8652

request
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      7180
           1       0.80      0.58      0.67      1472

    accuracy                           0.90      8652
   macro avg       0.86      0.78      0.81      8652
weighted avg       0.90      0.90      0.90      8652

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38

    accuracy                           1.00      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0.99      1.00      0.99      865


storm
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      7840
           1       0.72      0.68      0.70       812

    accuracy                           0.94      8652
   macro avg       0.84      0.83      0.83      8652
weighted avg       0.94      0.94      0.94      8652

fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8562
           1       0.53      0.18      0.27        90

    accuracy                           0.99      8652
   macro avg       0.76      0.59      0.63      8652
weighted avg       0.99      0.99      0.99      8652

earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      7865
           1       0.88      0.81      0.84       787

    accuracy                           0.97      8652
   macro avg       0.93      0.90      0.91      8652
weighted avg       0.97      0.97      0.97      86

c:\users\han\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\han\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\han\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
def save_pipeline(model_name,pipeline):
    url = '../models/'+ model_name +'.pickle'
    f = bz2.BZ2File(url, 'wb')
    pickle.dump(pipeline, f, protocol=2)
    f.close()

In [20]:
pipeline = pipeline_lgbClassifier()
parameters_grid =  {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.75, 1.0),
        'Multi_classifier__estimator__boosting_type' : ['dart'],
        'Multi_classifier__estimator__learning_rate' : (0.08,0.1)
        }


grid_cv = GridSearchCV(pipeline, parameters_grid, cv = 3,  verbose = 20, n_jobs=-1)

In [21]:
grid_cv.fit(X_train,y_train) 

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 18.9min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 26.8min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 28.0min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 28.3min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 28.4min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 36.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 37.4min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 37

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

In [24]:
pipeline = grid_cv.best_estimator_
save_pipeline('pipeline_optimized_model',pipeline)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
def compare_scores(pred_previous,pred_opimized):
    print('Previous Accuracy: {}'.format(accuracy_score(y_test,pred_previous)))
    print('New Accuracy: {}'.format(accuracy_score(y_test,pred_opimized)))
    print('Previous Precision: {}'.format(precision_score(y_test,pred_previous,average='macro')))
    print('New Precision: {}'.format(precision_score(y_test,pred_opimized,average='macro')))
    print('Previous Recall: {}'.format(recall_score(y_test,pred_previous,average='macro')))
    print('New Recalll: {}'.format(recall_score(y_test,pred_opimized,average='macro')))


In [14]:
def save_pipeline(model_name,pipeline):
    url = '../models/'+ model_name +'.pickle'
    f = bz2.BZ2File(url, 'wb')
    pickle.dump(pipeline, f, protocol=2)
    f.close()

In [ ]:
def load_pipeline(model_name):
    url = '../models/'+ model_name +'.pickle'
    f = bz2.BZ2File(url, 'r')
    pipeline_load = pickle.load(f)
    f.close()
    return pipeline_load

In [9]:
pickle_in = open("../models/pipeline_previous_model.pickle","rb")
pipeline_baseline = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open("../models/pipeline_optimized_model.pickle","rb")
pipeline_optimized = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open("../models/pipeline_final_model.pickle","rb")
pipeline_final = pickle.load(pickle_in)
pickle_in.close()

In [24]:
save_pipeline('pipeline_final_model',pipeline_final)
save_pipeline('pipeline_optimized_model',pipeline_optimized)
save_pipeline('pipeline_previous_model',pipeline_baseline)

In [15]:
pipe_test = load_pipeline('pipeline_final_compressed')

In [11]:
pred_baseline = pipeline_baseline.predict(X_test) 
pred_optimized = pipeline_optimized.predict(X_test) 

In [12]:
compare_scores(pred_baseline,pred_optimized) 

Previous Accuracy: 0.30386037910309754
New Accuracy: 0.28895053166897827
Previous Precision: 0.6073486260414658
New Precision: 0.6126202591069742
Previous Recall: 0.3469752705600752
New Recalll: 0.3715158193623531


c:\users\han\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\han\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
def pipeline_newClassifier(vect,tfidf,clf):
    pipeline = Pipeline([
                    ('vect', vect),
                    ('tfidf', tfidf),
                    ('Multi_classifier',MultiOutputClassifier(clf))
    ])
    return pipeline

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier()
vert = pipeline_optimized.steps[0][1]
tfidf = pipeline_optimized.steps[1][1]
pipeline_ada = pipeline_newClassifier(vert,tfidf,ada_clf)

In [41]:
pipeline_ada.fit(X_train,y_train)
pred_ada = pipeline_ada.predict(X_test)
compare_scores(pred_baseline,pred_ada)
compare_scores(pred_optimized,pred_ada)

Previous Accuracy: 0.30386037910309754
New Accuracy: 0.23231622746185854
Previous Precision: 0.6073486260414658
New Precision: 0.5378931491672823
Previous Recall: 0.3469752705600752
New Recalll: 0.3533827384429123
Previous Accuracy: 0.28895053166897827
New Accuracy: 0.23231622746185854

c:\users\han\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\han\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Previous Precision: 0.6126202591069742
New Precision: 0.5378931491672823
Previous Recall: 0.3715158193623531
New Recalll: 0.3533827384429123


In [1]:
def pipeline_lgb_add_feature(vect,tfidf):
    pipeline = Pipeline([('features',FeatureUnion(
                          [('text_pipeline1', Pipeline([
                                              ('vect', vect),
                                              ('tfidf', tfidf),
                                              ('T_SVD',TruncatedSVD())])),
                           ('text_pipeline2', Pipeline([
                                              ('vect2',TfidfVectorizer()),
                                              ('T_SVD', TruncatedSVD()) ])) 
                          ])),
                           
                        ('Multi_classifier',MultiOutputClassifier(LGBMClassifier(device='gpu', boosting_type='dart',
                                                                                 gpu_platform_id = 0, gpu_device_id = 0)))
    ])
    return pipeline

In [75]:
pipeline_model_final = pipeline_lgb_add_feature(vert,tfidf)

In [76]:
pipeline_model_final.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline1',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  lo

In [77]:
pred_final = pipeline_model_final.predict(X_test)

In [78]:
compare_scores(pred_optimized,pred_final)

Previous Accuracy: 0.28895053166897827
New Accuracy: 0.3060564031437818
Previous Precision: 0.6126202591069742
New Precision: 0.6315994052626203
Previous Recall: 0.3715158193623531
New Recalll: 0.37395112185470597


c:\users\han\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\han\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [80]:
save_pipeline('pipeline_final_model',pipeline_model_final)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.